# Domain Modeling

[Book Code](https://github.com/cosmicpython/code/tree/chapter_01_domain_model_exercise)

* How can we model business practices with code in a way that is compatible with TDD?

## What is a Domain Model?

* *Domain* is the problem you're trying to solve
* A *domain model* is the mental map clients have of the problem
* Jargon arises naturally in a domain model

* **Read Domain-Driven Design by Eric Evans**
* **Read Implementing Domain-Driving Design by Vaughn Vernon**
* Main idea: Software is valuable when it is a useful model of a problem.

### Minimal Example
This is an example of notes we might take for a company

* A product is defined by a SKU.
* Customers place orders
* An order is IDed by an order reference and comprises multiple order lines, where each line contains a SKU and a quantity
* The purchasing department orders batches of stock. A batch of stock has a unique ID called a reference, an SKU, and a quantity
* We need to allocate order lines to batches. When we've allocated an order line to a batch, we need to send units from that batch to the customer's address.
* When we allocate x units of stock from a batch, the available quantity is reduced by x.
* We can't allocate to a batch if the available quantity is less than the quantity of the order line.
* We can't allocate the same line twice.
* If we allocate the same line twice, the quantity should remain unchanged
* Batches have an ETA if they are currently shipping. Else, they may be in warehouse stock.
* We prefer to allocate to warehouse stock.
* If there isn't enough warehouse stock, we prefer to allocate to batches with the smallest ETA

* The [chapter 1 code](https://github.com/hovey/pyschool/tree/master/src/apwp/code/chapter_01) does implements according to the above spec.
* Notice how our tests are named using domain jargon.
* This is ripe for BDD tbh.

## Value Objects

* `OrderLine` is implemented as a `NamedTuple`. This is an example of something that follows the *Value Object Pattern*
* A value object is something that contains data, but no identity. 
* They are typically immutable
* Because they don't have an identity, we can do operations with them.
* `dataclass` and `NamedTuple` are both examples of value objects

In [13]:
from dataclasses import dataclass
from typing import NamedTuple

class Name(NamedTuple):
    first: str
    last: str

@dataclass(frozen=True)
class Money:
    currency: str
    value: int
    
    def __add__(self, other):
        if self.currency == other.currency:
            return Money(currency=self.currency, value=self.value + other.value)
    
    
john = Name(first="john", last="doe")
jane = Name(first="jane", last="doe")
assert john != jane

fiver = Money(currency="usd", value=5)
tenner = Money(currency="usd", value=10)
assert fiver + fiver == tenner

## Entities

* Value objects don't have a unique identifier. They are completely determined by the data they contain.
* On the other hand, our implementation of `Batch` does have a unique identified. This makes it an `Entity`.

In [14]:
class Person:
    # A person is an entity. They are the same person, independent of whether their name changes
    def __init__(self, *, name: Name):
        self.name = name
        
john_doe = Person(name=john)